In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.model_selection import cross_validate, train_test_split, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler 
from scipy.stats import randint, loguniform

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)